In [ ]:
!pip install gdown jcopdl
!gdown https://drive.google.com/uc?id=1sGZwJdbk7ZWqKEORJp6blndHX8azDXkt


In [ ]:
# MNIST https://drive.google.com/uc?id=1sGZwJdbk7ZWqKEORJp6blndHX8azDXkt
# DCGAN https://drive.google.com/uc?id=1jOwPCFKsIoQR1a0cEiKNpwPuw_kEpXAh
# CelebA https://drive.google.com/uc?id=1KaiwyyYRGW8FbvSd4Feg1i1YW2k2s30u

#### di GAN ini trainingnya gak stabil, maka untuk menghindari itu jangan ada angka nol
* lakukan normalize
* jangan gunakan relu (karna relu bisa menyebabkan angka nol)

In [2]:
import torch
from torch import nn,optim

In [3]:
device= torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

# Dataset dan Dataloader ~> hanya data train

In [4]:
from torch.utils.data import DataLoader
from torchvision import datasets,transforms

In [6]:
bs = 64

transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5]) # normalisasi agar stabil dan menjadi [-1,1]
])

trainset=datasets.ImageFolder("data/train/",transform=transform)
trainloader = DataLoader(trainset,bs,shuffle=True, num_workers=4)

# Arsitektur dan Config
* arsitektur nya kita buat 2, karna ada discriminator dan generator
* coba tulis menjadi script agar gampang import import, syarat nya import lagi apa aja yang dibutuhin

In [29]:
%%writefile model_gan2.py 
# ini cara agar di tulis jadi script yang nanti nya bisa di import

import torch
from torch import nn, optim
from jcopdl.layers import linear_block

class Disciminator (nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Flatten(),
            linear_block(784,512, activation = "lrelu"), # activation ini untuk menghindari angka nol
            linear_block(512,256, activation = "lrelu"),
            linear_block(256,128, activation = "lrelu"),
            linear_block(128,1, activation = "sigmoid")
        )
    def forward(self,x):
        return self.fc(x)
    
class Generator (nn.Module):
    def __init__(self, n_latent):
        super().__init__()
        self.n_latent = n_latent
        self.fc = nn.Sequential(
            linear_block(n_latent,128, activation = "lrelu"),
            linear_block(128,256, activation = "lrelu", batch_norm = True), # activation ini untuk menghindari angka nol
            linear_block(256,512, activation = "lrelu", batch_norm = True),
            linear_block(512,1024, activation = "lrelu", batch_norm = True),
            linear_block(1024,784, activation = "tanh") # karna tadi di generator input gambar nya dari -1 sampe 1 maka nya generate fake image harus menyesuaikan juga
        )
    def forward(self,x):
        return self.fc(x)
    
    # buat satu fungsi untuk generate n(seberapa banyak) ukuran latent space atau random number seukuran dengan n_latent
    def generate (self, n, device):
        z= torch.randn((n, self.n_latent), device=device) # generate random number sebanyak n kali latent space kita atau generate n gambar palsu
        return self.fc(z) # dari random number ini kita masukin ke arsitektur generator kita untuk di generate gambar

Writing model_gan2.py


In [9]:
from jcopdl.callback import set_config

In [10]:
config = set_config({
    "n_latent" : 100,
    "batch_size": bs
})

# Training Preparation ~> MCO
* disini tanpa ada nya Callback karna model kita akan berkompetisi (generator dengan discriminator) sampe ke kedua nya sama sama pintar (nash equilibrium)
* jadi kita akan menentukan mau seberapa banyak epoch nya. gunain konsep bayes varians trade off (kalau masih kurang ya training ulang dengan epoch yang lebih lama)
* optimizer nya kita jangan gunain adam weight karna nanti bisa terkena mode collaps dimana antara generator atau discriminator kita tidak saling berkompetisi (maksudnya baru training awal awal saja sudah ada yang tidak training) ~> maka gunain adam biasa dan learning rate nya kecil aja biar model jalan nya pelan pelan

In [30]:
from model_gan2 import Disciminator,Generator

In [34]:
D = Disciminator().to(device)
G = Generator(config.n_latent).to(device)

criterion = nn.BCELoss()

d_optimizer = optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = optim.Adam(G.parameters(), lr=0.0002)

# Training 
* kita manual (seperti dulu training di nn) karna gak pakai Callback 
* disini kita juga akan generate real n fake images (karna nanti ini yang sebagai data input)

* disini kita juga akan generate real n fake labels (karna kita gak gunain label asli kita)

* disini kita juga akan training Discriminator ~> 
    - reset dulu zero_grad dari optimizer(reset gradien),
    - feedforward aja data real dan data fake kita, tapi jangan lupa data fake kita di detach(diputus jalur ke generator nya)

* disini kita juga akan training Generator ~>

In [37]:
import os
os.makedirs("output/GAN/",exist_ok=True)
os.makedirs("model/GAN/",exist_ok=True)

from torchvision.utils import save_image # untuk hasil generate fake image ditampilin seperti colase

In [ ]:
epochs = 500
for epoch in range(epochs):
    D.train()
    G.train()
    for real_img, _ in trainloader:
        n_data = real_img.shape[0] #mastiin bahwa jumlah data yang diambil sebanyak 1 batch sama dengan 64
        
        # generate real n fake images
        real_img = real_img.to(device)
        fake_img = G.generate(n_data,device)
        # real n fake labels
        real = torch.ones((n_data, 1), device=device)
        fake = torch.zeros((n_data, 1),device=device)
        
        # training Discriminator (hanya feedforward, hitung loss, bacprop, update weight)
        d_optimizer.zero_grad() # buat mastiin aja tiap looping selalu reset dulu
        # real image ~> Discriminator ~> label real
        output = D(real_img) #feedforward
        d_real_loss = criterion(output, real) # hitung loss
        
        # fake image ~> Discriminator ~> label fake
        output = D(fake_img.detach()) #feedforward
        d_fake_loss = criterion(output, fake) #hitung loss
        
        d_loss = d_real_loss+d_fake_loss # gabungin loss buat di hitung backprop
        d_loss.backward() # backprop
        d_optimizer.step() #update weight
        
        # training Generator
        g_optimizer.zero_grad() # buat mastiin aja tiap looping selalu reset dulu
        #Fake image ~> Discriminator ~> tapi label nya real
        output = D(fake_img) #feedforward dari G.generate = fake image
        g_loss = criterion(output, real) #hitung loss
        
        g_loss.backward() # backprop
        g_optimizer.step() #update weight
        
        
    if epoch % 5 == 0:
        print(f"Epoch:{epoch} | D_loss:{d_loss/2:.5f} | G_loss : {g_loss:.5f}")
    
    if epoch % 15 == 0:   #tiap epoch yang habis dibagi 15 akan digenerate fake img nya, ya inti nya buat mantau model kita udah bisa generate fake image yang bagus apa belum
        G.eval()
        epoch=str(epoch).zfill(4) # untuk jadiin epoch nya string lalu zfill untuk menjadikan ada berapa angka yang bisa ditampung ex zfill 4 maka epoch ke 5 sama dengan 0005
        fake_img = G.generate(64, device=device)
        save_image(fake_img.view(64,1,28,28),f"output/GAN/{epoch}.jpg",nrow=8,normalize=True)
        
        torch.save(D,"model/GAN/discriminator.pth")
        torch.save(G,"model/GAN/generator.pth") # kita bisa save model nya (bukan hanya weight saja), tapi dengan syarat arsitektur kita udah dalam bentuk script (.py) karna arsitektur kita sudah terbungkusd dan bisa di akses dimana pun
        